In [75]:
import os
os.environ['HF_TOKEN'] = 'ENTER YOUR TOKEN HERE'

In [76]:
import PyPDF2
import math
import os
import requests

pdf_path = "/content/test.pdf"
full = ""

with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    for page in reader.pages:
        text = page.extract_text() or ""
        full += text + " "

words = full.split()
x = math.ceil(len(words) / 50)
setofwords = [" ".join(words[i * 50:(i + 1) * 50]) for i in range(x)]

print(len(setofwords))



15


In [77]:
question = input("Enter your question: ")

Enter your question: what is AI ?


In [80]:
import torch
from sentence_transformers import SentenceTransformer

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model_id = "google/embeddinggemma-300M"
model = SentenceTransformer(model_id)

# Encode the sentences on the chosen device
pdfembeddings = model.encode(
    setofwords,
    device=device,      # Move computation to GPU if available
    batch_size=16,      # Adjust depending on memory
    show_progress_bar=True
)

questionembedding = model.encode(question, device=device)

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# pdfembeddings: (num_sentences, embedding_dim)
# questionembedding: (embedding_dim,) → reshape to (1, embedding_dim)
question_vec = questionembedding.reshape(1, -1)

# Compute cosine similarity between question and all PDF embeddings
similarities = cosine_similarity(question_vec, pdfembeddings)  # shape: (1, num_sentences)

# Flatten the result
similarities = similarities.flatten()

# Find the index of the best matching sentence
best_idx = np.argmax(similarities)
best_score = similarities[best_idx]
best_context = setofwords[best_idx]

context = "You are a teacher and your job is to help your student easily answer the question he asked using the context provided -> context = "
context = context + best_context
context = context + " -> question = "
context = context + question
print(context)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You are a teacher and your job is to help your student easily answer the question he asked using the context provided -> context = of the key ways in which AI is transforming education is through personalized learning . Traditional classrooms often follow a one -size-fits-all approach, where every student is expected to learn at the same pace and in the same manner. This method leaves behind students who learn more slowly, while failing -> question = what is AI ?


In [81]:
import os
import requests

API_URL = "https://router.huggingface.co/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {os.environ['HF_TOKEN']}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

response = query({
    "messages": [
        {
            "role": "user",
            "content": context
        }
    ],
    "model": "openai/gpt-oss-120b:cerebras"
})

print(response["choices"][0]["message"])

{'content': '**Answer:**\n\nArtificial Intelligence (AI) is a branch of computer science that creates machines and software able to **think, learn, and make decisions** like a human does. Instead of just following fixed instructions, AI systems can analyze data, recognize patterns, and improve their performance over time.\n\n**Why this matters for education (from the context you gave):**  \nAI lets computers understand each student’s unique strengths, weaknesses, and learning speed. By doing so, it can provide **personalized learning experiences**—for example, giving extra practice to a student who needs it, or offering more challenging material to someone who’s ready for it. This is very different from the traditional “one‑size‑fits‑all” classroom, where every student is expected to learn in the same way and at the same pace.  \n\nSo, in short, AI is technology that can mimic human intelligence, and in education it helps create customized learning paths for every learner.', 'reasoning